# R Custom Language Wrapper with Docker Build

 * Wrap an R decision-tree model (using the ```rpart``` library) for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy with Seldon Deploy
 
## Dependencies

 * Docker
 * R

```bash
pip install seldon-core
```

## Train locally
 

In [ ]:
!Rscript train.R

In [ ]:
!mv model.Rds deployment/model.Rds

## Build docker image

In [ ]:
!docker build deployment/. -t r-iris:0.1

## Test Locally

Open a terminal and run the following command to run the microservice:

`!docker run --name "r-iris" -p 5000:5000 r-iris:0.1`

Use the Seldon Core python client to generate and send a request to the microservice:

In [ ]:
!seldon-core-tester contract.json 127.0.0.1 5000 -p

## Push to Dockerhub

List all the running containers in docker engine to find the `CONTAINER ID`

In [ ]:
!docker ps

In [ ]:
!docker commit <CONTAINER ID> r-iris

Tag the image including your Docker Hub `USERNAME` and version, then push the image:

In [ ]:
!docker tag r-iris <USERNAME>/r-iris:0.1
!docker push <USERNAME>/r-iris:0.1

## Deploy with Seldon Deploy

If you have access to a cluster running Seldon Deploy you can create a deployment from the container following these steps:

```bash
pip install seldon-deploy-sdk
```

In [ ]:
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

Authenticate with cluster: 

In [ ]:
SD_IP = ""
username = ""
password = ""

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"

config.oidc_client_id = "sd-api"
config.oidc_client_secret = "sd-api-secret"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"

def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate(username, password)
    api_client = ApiClient(config)
    return api_client

Create the deployment manifest remembering to provide the location of the image for `MODEL_LOCATION`:

In [ ]:
DEPLOYMENT_NAME = "r-iris"
NAMESPACE = "test"
MODEL_LOCATION = ""

CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "image": MODEL_LOCATION,
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

## Further Testing

Once the model is deployed, you can test it by sending the following data payload in a request:

`{
   "data": {
	"names": ["sepal_length","sepal_width","petal_length", "petal_width"],
	"ndarray": [
	    [6.8,  2.8,  4.8,  1.4]
	]
   }
}`